## Calcular Métricas - Tempo de Votação, Biometria, etc.

---

## Importing libraries

In [7]:
import duckdb
import pandas as pd
import time

## Importing Data

In [8]:
cursor = duckdb.connect()

Dados Brutos

In [9]:
TABLE = "read_parquet('VOTES.parquet/*/*/*/*.parquet', hive_partitioning=True)"
ZONE_GROUPS = [ (x, x+20) for x in range(0, 800, 20) ]

Adicionar TURNO e Timestamp final de Biometria

In [4]:
source = F"""
(
    SELECT 
        *,
        
        CASE event_date
            WHEN '2022-10-02' THEN 1
            WHEN '2022-10-03' THEN 1
            WHEN '2022-10-30' THEN 2
            WHEN '2022-10-31' THEN 2
            ELSE NULL
        END::INT AS turno,

        COALESCE(
            timestamp_biometria_1,
            timestamp_biometria_2,
            timestamp_biometria_3,
            timestamp_biometria_4,
            timestamp_biometria_manual
        ) AS timestamp_biometria_final,

        strftime( '%Y-%m-%d %H:', timestamp_voto_computado )
        || (EXTRACT(MINUTE FROM timestamp_voto_computado)//5)*5 + 5
        || ':00' AS timestamp_voto_computado_5min
        
    FROM 
        {TABLE}
    -- WHERE uf='DF'
) _
"""

## Preparinga Data

Méticas no Cubo OLAP - Turno, UF, Zona, Seção.

- Número de Votos
- Número de Seções Eleitorais
- Média, Soma, q50%, q90% do Tempo total de Voto, Tempo de Biometria, Tempo Total

- Quantidade de Votos efetuados em até 30s, 1min, 1min30s, 2min, 2min30s, 3min+
- Taxa de Sucesso da Biometria em 1 tentativa, 2 tentativas, 3 tentativas, 4 tentativas, Falha
- Quantidade de Teclas Pressionadas
- Quantidade de Cargos Distintos Votados

**Definição das métricas de tempo**

In [5]:
tempo_voto_total = "EXTRACT(EPOCH FROM (timestamp_voto_computado  - timestamp_titulo_digitado))"
tempo_voto       = "EXTRACT(EPOCH FROM (timestamp_voto_computado  - timestamp_habilitacao_eleitor))"
tempo_biometria  = "EXTRACT(EPOCH FROM (timestamp_biometria_final - timestamp_titulo_digitado))"

fix_null_values = lambda column: F"COALESCE({column}::VARCHAR(20), 'ALL')"

In [6]:
query_metrics = F"""
    SELECT
        {fix_null_values('turno') } AS turno,
        {fix_null_values('timestamp_voto_computado_5min') } AS timestamp_voto_computado_5min,
        {fix_null_values('uf') } AS uf,
        zone_code,
        {fix_null_values('section_code') } AS section_code,

        COUNT(*) AS total_votos,
        SUM( {tempo_voto} ) AS tempo_voto_soma,
        SUM( {tempo_biometria} ) AS tempo_biometria_soma,
        SUM( {tempo_voto_total} ) AS tempo_voto_total_soma,
        
    FROM
        {source}
    WHERE quantidade_votos_computados = 1
    GROUP BY ROLLUP(turno, timestamp_voto_computado_5min, uf, zone_code, section_code)
"""

**Salvar resultado intermediário**

In [7]:
query = F"""
    COPY (
    {
        query_metrics
    } )
    TO 'VOTES_TIME_METRICS_OVER_TIME.parquet' 
    (FORMAT 'parquet', PARTITION_BY (turno, uf), OVERWRITE_OR_IGNORE 1);
"""

In [8]:
cursor.execute(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cálculo cumulativo das métricas

In [11]:
TABLE_METRICS = """read_parquet(
    'VOTES_TIME_METRICS_OVER_TIME.parquet/*/*/*.parquet', 
    hive_partitioning=True,
    hive_types_autocast =0
)
"""

In [12]:
query_cumulative_metrics = F"""
    SELECT
        *,
        SUM(total_votos) OVER (PARTITION BY turno, uf, zone_code, section_code ORDER BY timestamp_voto_computado_5min) AS total_votos_cumulativo,
        SUM(tempo_voto_soma) OVER (PARTITION BY turno, uf, zone_code, section_code ORDER BY timestamp_voto_computado_5min) AS tempo_voto_soma_cumulativo,
        SUM(tempo_biometria_soma) OVER (PARTITION BY turno, uf, zone_code, section_code ORDER BY timestamp_voto_computado_5min) AS tempo_biometria_soma_cumulativo,
        SUM(tempo_voto_total_soma) OVER (PARTITION BY turno, uf, zone_code, section_code ORDER BY timestamp_voto_computado_5min) AS tempo_voto_total_soma_cumulativo
    FROM
        {TABLE_METRICS}
"""

Os arquivos parquet são particionados por DATA DO EVENTO, UF e GRUPO DE ZONA ELEITORAL para agilizar a leitura dos dados pelo Dashboard.

As ZONAS foram agrupadas em grupos de 20, esse número é empírico.

In [13]:
query_metrics_with_zone_group = F"""
    SELECT
    *,
    CASE
        {
            "".join(
                [
                    f"WHEN zone_code IS NOT NULL AND zone_code::INT BETWEEN {min_zone} AND {max_zone} THEN '{min_zone}-{max_zone}' " 
                    for min_zone, max_zone in ZONE_GROUPS
                ]
            )
        }
        ELSE 'ALL'
    END AS zone_group
    FROM (
        {query_cumulative_metrics}
    ) _
"""

In [14]:
query = F"""
    COPY (
    {
        query_metrics_with_zone_group
    } )
    TO 'VOTES_TIME_CUMULATIVE_METRICS_OVER_TIME.parquet' 
    (FORMAT 'parquet', PARTITION_BY (turno, uf, zone_group), OVERWRITE_OR_IGNORE 1);
"""

In [15]:
cursor.execute(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))